In [2]:
import os
import netCDF4 as nc
import geopandas as gpd
import rasterio
from osgeo import gdal, osr
from rasterstats import zonal_stats

# 1. Convert NC to raster

In [18]:
def nc_to_geotiff(read_path, save_path):
    nc_file = nc.Dataset(read_path)
    lat = nc_file.variables['latitude'][:]
    lon = nc_file.variables['longitude'][:]
    PM25 = np.asanyarray(nc_file.variables['PM25'])

    # get the spatial range of the netcdf
    Lonmin, Latmax, Lonmax, Latmin = [lon.min(), lat.max(), lon.max(), lat.min()]

    # calculate the resolution
    Num_lat = len(lat)
    Num_lon = len(lon)
    Lat_res = (Latmax - Latmin) / (float(Num_lat) - 1)
    Lon_res = (Lonmax - Lonmin) / (float(Num_lon) - 1)

    # create the tif file and save it into the virtual file system in memory
    driver = gdal.GetDriverByName('GTiff')
    out_tif = driver.Create('/vsimem/PM25.tif', Num_lon, Num_lat, 1, gdal.GDT_Float32)

    # set the spatial range of the tif file
    geotransform = (Lonmin, Lon_res, 0.0, Latmax, 0.0, -Lat_res)
    out_tif.SetGeoTransform(geotransform)

    # set the projection system
    prj = osr.SpatialReference()
    prj.ImportFromEPSG(4326)
    out_tif.SetProjection(prj.ExportToWkt())

    # check is the data are flipped and correct the data if yes
    if lat[0] <= lat[-1]: 
        PM25 = PM25[::-1]
    else:
        pass

    # write data into tif and close the file
    out_tif.GetRasterBand(1).WriteArray(PM25)
    # transform the projection to 3035 and save
    gdal.Warp(save_path, out_tif, srcSRS='EPSG:4326', dstSRS='EPSG:3035')
    out_tif.FlushCache() 
    out_tif = None

In [19]:
read_path = r'D:\OneDrive - Nexus365\1-Oxford\1 - Oxford Uni related\7 - Services\2024.05.20 Mapineq\0-data\PM2.5\V6GL02-a.CNNPM25.Global.201901-201912.nc'
save_path = r'D:\OneDrive - Nexus365\1-Oxford\1 - Oxford Uni related\7 - Services\2024.05.20 Mapineq\0-data\PM2.5\V6G02_2019.tif'
if __name__ == '__main__':
    nc_to_geotiff(read_path, save_path)

# 2. Zonal statistics

In [3]:
# read nuts shp and PM2.5 raster data
nuts = gpd.read_file(r'D:\OneDrive - Nexus365\1-Oxford\1 - Oxford Uni related\7 - Services\2024.05.20 Mapineq\0-data\NUTS\NUTS_RG_01M_2021_3035\NUTS_RG_01M_2021_3035.shp')
PM_ras = rasterio.open(r'D:\OneDrive - Nexus365\1-Oxford\1 - Oxford Uni related\7 - Services\2024.05.20 Mapineq\0-data\PM2.5\V6G02_2019.tif')

In [ ]:
zonal_stats(nuts, PM_ras, stats='mean')